# Draw nodal points
This script finds pairs of column vectors with different sign for each eigenvector and draws a magnitude-weighted point between the column vectors.

## Purpose
Visualize nodal surfaces for each eigenvector.

## Methodology
Each normal mode contains one or more nodal surfaces where bead displacemnet is zero.

## WIP - improvements
Use this section only if the notebook is not final.

Notable TODOs:
- todo 1;
- todo 2;
- todo 3.

## Results
Describe and comment the most important results.

## Suggested next steps
State suggested next steps, based on results obtained in this notebook.

# Setup

## Library import
We import all the required Python libraries

In [1]:
# Data manipulation
import pandas as pd
import numpy as np
import os
import glob
from biopandas.pdb import PandasPdb
from pymol import cmd

# Options for pandas
pd.options.display.max_columns = 50
pd.options.display.max_rows = 30

# Visualizations
import plotly
import plotly.graph_objs as go
import plotly.offline as ply
plotly.offline.init_notebook_mode(connected=True)

import matplotlib.pyplot as plt
import seaborn as sns

# Autoreload extension
if 'autoreload' not in get_ipython().extension_manager.loaded:
    %load_ext autoreload
    
%autoreload 2

### Change directory
If Jupyter lab sets the root directory in `notebooks`, change directory.

In [2]:
if "notebook" in os.getcwd():
    os.chdir("..")

## Local library import
We import all the required local libraries libraries

In [3]:
# Include local library paths
import sys
sys.path.append("./src") # uncomment and fill to import local libraries

# Import local libraries
import src.utilities as utils

# Parameter definition
We set all relevant parameters for our notebook. By convention, parameters are uppercase, while all the 
other variables follow Python's guidelines.

In [4]:
EIGENVECTOR_FILEPATH = "data/processed/eigenvector_024"
config = utils.read_config()

COORDS = "data/raw/CAonly.pdb"

plt.style.use(config['viz']['jupyter'])


# Data import
We retrieve all the required data for the analysis.

In [5]:
eigenvector_filepaths = sorted(glob.glob("data/processed/eigenvector_???"))
no_modes = len(eigenvector_filepaths)
eigenvector_filepath_format = "data/processed/eigenvector_{:03}"
eigenvectors = {mode_num : np.loadtxt(eigenvector_filepath_format.format(mode_num)) for mode_num in range(7,no_modes+7)}

coords = PandasPdb().read_pdb(COORDS)

# Data processing
Put here the core of the notebook. Feel free to further split this section into subsections.

### Renormalize each row/column vector of mode eigenvector
**Note** The motion's amplitude inforamtion is lost.

In [6]:
# Renormalize eigenvectors
# for key in eigenvectors.keys():
#     eigenvector = eigenvectors[key]
#     for row_idx in range(eigenvector.shape[0]):
#         eigenvector[row_idx] = eigenvector[row_idx] / np.linalg.norm(eigenvector[row_idx])

In [7]:
coords = coords.df['ATOM'][['x_coord','y_coord','z_coord']].to_numpy()

In [23]:
def calcualte_dotprod(eigenvector):
    no_beads = np.shape(eigenvector)[0]
    dotprod = np.zeros((no_beads, no_beads))

    for i in np.arange(no_beads):
        for j in np.arange(i+1):
            vector_1 = eigenvector[i]
            vector_2 = eigenvector[j]
            dotprod[i][j] = np.dot(vector_1, vector_2)

    dotprod = dotprod + dotprod.T
    return dotprod

def calcualte_crossprod_norm(eigenvector):
    no_beads = np.shape(eigenvector)[0]
    crossprod_norm = np.zeros((no_beads, no_beads))

    for i in np.arange(no_beads):
        for j in np.arange(i+1):
            vector_1 = eigenvector[i]
            vector_2 = eigenvector[j]
            crossprod_norm[i][j] = np.linalg.norm(np.cross(eigenvector[i], eigenvector[j]))

    crossprod_norm = crossprod_norm + crossprod_norm.T
    return crossprod_norm

def factorize(number):
    """ Factorizes number into two near integers.
    """
    x = math.floor(math.sqrt(number))
    y = math.ceil(number / x)
    return (x, y)

def find_nodal_points(coords, eigenvector, neg_dotprod_idxs):
    nodal_points = []
    for residue_pair in neg_dotprod_idxs:
        residue_idx_1 = residue_pair[0]
        residue_idx_2 = residue_pair[1]

        pos_vector_1 = coords[residue_idx_1]
        pos_vector_2 = coords[residue_idx_2]

        diff_vector = pos_vector_2 - pos_vector_1
        eigenvector_mag_1 = np.linalg.norm(eigenvector[residue_idx_1])
        eigenvector_mag_2 = np.linalg.norm(eigenvector[residue_idx_2])

        nodal_point = diff_vector * (eigenvector_mag_1 / (eigenvector_mag_1 + eigenvector_mag_2)) + pos_vector_1
        nodal_points.append(nodal_point)
    
    return nodal_points

In [21]:
eigenvector = eigenvectors[7]

dotprod = calcualte_dotprod(eigenvector)
# Mask upper triangualr part
dotprod[np.mask_indices(dotprod.shape[0], np.triu)] = 0

neg_dotprod_idxs = np.where(dotprod < 0)
neg_dotprod_idxs = [(neg_dotprod_idxs[0][i],neg_dotprod_idxs[1][i]) for i in range(len(neg_dotprod_idxs[0]))]

nodal_points = []
for residue_pair in neg_dotprod_idxs:
    residue_idx_1 = residue_pair[0]
    residue_idx_2 = residue_pair[1]

    pos_vector_1 = coords[residue_idx_1]
    pos_vector_2 = coords[residue_idx_2]
    
    print("pos_vector_{}".format(residue_pair[0]))
    print("{:.3f} {:.3f} {:.3f}".format(*pos_vector_1))
    print("pos_vector_{}".format(residue_pair[1]))
    print("{:.3f} {:.3f} {:.3f}".format(*pos_vector_2))
    print("pos_vector_{} - pos_vector_{}".format(residue_pair[1], residue_pair[0]))
    diff_vector = pos_vector_2 - pos_vector_1
    print("{:.3f} {:.3f} {:.3f}".format(*diff_vector))
    eigenvector_mag_1 = np.linalg.norm(eigenvector[residue_idx_1])
    eigenvector_mag_2 = np.linalg.norm(eigenvector[residue_idx_2])
    nodal_point = diff_vector * (eigenvector_mag_1 / (eigenvector_mag_1 + eigenvector_mag_2)) + pos_vector_1
    print("{:.3f} {:.3f} {:.3f}".format(*nodal_point))
    nodal_points.append(nodal_point)
    print()

pos_vector_1
4.000 0.000 0.000
pos_vector_0
0.000 0.000 0.000
pos_vector_0 - pos_vector_1
-4.000 0.000 0.000
2.000 0.000 0.000

pos_vector_3
2.000 1.155 3.266
pos_vector_2
2.000 3.464 0.000
pos_vector_2 - pos_vector_3
0.000 2.309 -3.266
2.000 2.309 1.633



# Data visualization
Visualize processed data

In [24]:
# Draw nodal points
structure_filepath = "data/raw/CAonly.pdb"
for mode_num, eigenvector in eigenvectors.items():
    dotprod = calcualte_dotprod(eigenvector)
    
    neg_dotprod_idxs = np.where(dotprod < 0)
    neg_dotprod_idxs = [(neg_dotprod_idxs[0][i],neg_dotprod_idxs[1][i]) for i in range(len(neg_dotprod_idxs[0]))]
    
    nodal_points = find_nodal_points(coords, eigenvector, neg_dotprod_idxs)
    cmd.reinitialize()
    structure_name ="CAonly"
    pymol_view =    ((\
                    0.254109710,    0.685759246,    0.682028472,\
                    -0.659577310,   -0.392885894,    0.640778780,\
                    0.707380295,   -0.612679243,    0.352475613,\
                    0.000000893,   -0.000001701,  -20.039798737,\
                    2.060348749,    0.924048603,    0.750656426,\
                    17.039802551,   23.039796829,  -20.000000000 ))
    cmd.delete('all')
    cmd.load(structure_filepath, structure_name)
    cmd.set('sphere_scale', 0.5, structure_name)
    for idx, nodal_point in enumerate(nodal_points):
        nodal_point = nodal_point.tolist()
        cmd.pseudoatom("nodal_points", selection='', name='PS1', resn='NDP', resi=str(idx+1), chain='A',
                       segi='PSDO', elem='PS', vdw=-1.0, hetatm=1, b=0.0, q=0.0, color='tv_red',
                       label='', pos=nodal_point, state=0, mode='rms', quiet=1)

    cmd.show_as('sphere', 'all')
    cmd.set('sphere_scale', 0.3, "nodal_points")

    cmd.viewport(width=1200, height=1200)
    cmd.zoom(buffer=1,complete=1)

    cmd.save("tmp/mode_{:03}.pse".format(mode_num))
    cmd.set('ray_opaque_background', 0)
    cmd.png("tmp/mode_{:03}.png".format(mode_num), width=1200, height=1200, ray=1)

In [ ]:
with plt.rc_context({'font.size': 4, 'xtick.labelsize': 5, 'ytick.labelsize': 5, 'axes.titlesize': 7}):
    _, axs = plt.subplots(subplot_rows, subplot_columns)
    mode_num = 7
    for i in range(subplot_rows):
        for j in range(subplot_columns):
            corossprod_norm = calcualte_crossprod_norm(eigenvectors[mode_num])
            plot_crosscor(corossprod_norm, vmin=0, center=None, vmax=1, cmap=plt.cm.binary, axis=axs[i, j], annot=False)

            axs[i, j].set_title("mode {}".format(mode_num))
            mode_num += 1

    plt.tight_layout()

# References
We report here relevant references:
1. author1, article1, journal1, year1, url1
2. author2, article2, journal2, year2, url2